# Shifty Lines: Line Detection and Doppler Shift Estimation

We have some X-ray spectra that have absorption and emission lines in it. The original spectrum is seen through a stellar wind, which moves either toward or away from us, Doppler-shifting the absorbed lines. Not all lines will be absorbed, some may be at their original position. There may also be more than one redshift in the same spectrum.
There are various other complications: for example, we rarely have the complete spectrum, but separate intervals of it (due to issues with calibration). In principle, however, the other segments may give valuable information about any other given segment.


Simplest problem: estimating Doppler shift and line presence at the same time

Second-simplest problem: some lines are Doppler shifted, some are not

Full problem: estimating the number of redshifts and the lines belonging to each in the same model

Note: we are going to need to add some kind of OU process or spline to model the background.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy.special

from astropy import units
import astropy.constants as const

import emcee
import corner

Let's first load our first example spectrum:

In [ ]:
datadir = "/Users/danielahuppenkothen/work/data/spectral_data/shifty_lines/"
filename = "8525_nodip_0.dat"

In [ ]:
data = np.loadtxt(datadir+filename)

In [ ]:
wavelength_left = data[:,0]
wavelength_right = data[:,1]

wavelength_mid = data[:,0] + (data[:,1]-data[:,0])/2.
counts = data[:,2]
counts_err = data[:,3]

In [ ]:
wavelength_mid[::-1]

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(wavelength_mid, counts)
plt.gca().invert_xaxis()
plt.xlim(wavelength_mid[-1], wavelength_mid[0])

Next, we're going to need the lines we're interested in. Let's use the Silicon lines. Note that these are all in *electron volt*. However, the data are in *Angstrom*, which means I need to convert them. 

I'm going to use `astropy.units` to do that:

In [ ]:
siXIV = 1864.9995 * units.eV
siXIII = 2005.494 * units.eV

siXII = 1845.02 * units.eV
siXII_err = 0.07 * units.eV

siXI = 1827.51 * units.eV
siXI_err = 0.06 * units.eV

siX = 1808.39 * units.eV
siX_err = 0.05 * units.eV

siIX = 1789.57 * units.eV
siIX_err = 0.07 * units.eV

siVIII = 1772.01 * units.eV
siVIII_err = 0.09 * units.eV

siVII = 1756.68 * units.eV
siVII_err = 0.08 * units.eV

si_all = [siXIV, siXIII, siXII, siXI, siX, siIX, siVIII, siVII]
si_err_all = [0.0*units.eV, 0.0*units.eV, siXII_err, siXI_err, siX_err, siIX_err, siVIII_err, siVII_err]

Now I can do the actual conversion:

In [ ]:
si_all_angstrom = [(const.h*const.c/s.to(units.Joule)).to(units.Angstrom) 
                   for s in si_all]
si_err_all_angstrom = [(const.h*const.c/s.to(units.Joule)).to(units.Angstrom) 
                       for s in si_err_all]

si_err_all_angstrom[0] = 0.0*units.Angstrom
si_err_all_angstrom[1] = 0.0*units.Angstrom

In [ ]:
for s in si_all_angstrom:
    print(s)

Let's plot the lines onto the spectrum:

In [ ]:
plt.figure(figsize=(16,4))
plt.errorbar(wavelength_mid, counts, yerr=counts_err, fmt="o")
plt.gca().invert_xaxis()
plt.xlim(wavelength_mid[-1], wavelength_mid[0])
for s in si_all_angstrom:
    plt.vlines(s.value, np.min(counts), np.max(counts), lw=2)

We currently don't have the positions of the longer-wavelength lines, so we're going to cut the spectrum at 7.1 Angstrom:

In [ ]:
maxind = wavelength_mid.searchsorted(7.1)

In [ ]:
wnew_mid = wavelength_mid[:maxind]
wnew_left = wavelength_left[:maxind]
wnew_right = wavelength_right[:maxind]

cnew = counts[:maxind]
enew = counts_err[:maxind]

In [ ]:
cnew.shape

In [ ]:
plt.figure(figsize=(16,4))
plt.errorbar(wnew_mid, cnew, yerr=enew, fmt="o")
plt.gca().invert_xaxis()
plt.xlim(wnew_mid[-1], wnew_mid[0])
for s in si_all_angstrom:
    plt.vlines(s.value, np.min(counts), np.max(counts), lw=2)

We are going to save the spectrum as well as the line centers:

In [ ]:
np.savetxt(datadir+"8525_nodip_cut.txt", np.array([wnew_left, wnew_right, wnew_mid, cnew, enew]).T)

In [ ]:
## convert from astropy.units to float
si_all_val = np.array([s.value for s in si_all_angstrom])
si_err_all_val = np.array([s.value for s in si_err_all_angstrom])

In [ ]:
np.savetxt(datadir+"si_lines.txt", np.array([si_all_val, si_err_all_val]).T)

## Simulating Data

In order to test any methods we are creating, we are first going to produce some simulated data. 

For a start, we are simulating two types of data sets: one where the lines are *all* Doppler shifted by a certain amount, and one where a random subset of lines is Doppler shifted.

In [ ]:
np.random.seed(20160216)

In [ ]:
## shift in the line centroid

dshift = np.random.normal(loc=0.0, scale=0.2)
print(dshift)

In [ ]:

def gaussian_cdf(x, w0, width):
    return 0.5*(1. + scipy.special.erf((x-w0)/(width*np.sqrt(2.))))

def spectral_line(wleft, wright, w0, amplitude, width):
    """
    Use the CDF of a Gaussian distribution to define spectral 
    lines. We use the CDF to integrate over the energy bins, 
    rather than taking the mid-bin energy.
    
    Parameters
    ----------
    wleft: array
        Left edges of the energy bins
        
    wright: array
        Right edges of the energy bins
        
    w0: float
        The centroid of the line
        
    amplitude: float
        The amplitude of the line
        
    width: float
        The width of the line
        
    Returns
    -------
    line_flux: array
        The array of line fluxes integrated over each bin
    """
    line_flux = amplitude*(gaussian_cdf(wright, w0, width)-
                           gaussian_cdf(wleft, w0, width))
    return line_flux

In [ ]:
w0 = 6.6
amp = 0.01
width = 0.01

line_flux = spectral_line(wnew_left, wnew_right, w0, amp, width)


In [ ]:
plt.plot(wnew_mid, line_flux)

Let's fake some data.

Below is a function that can simulate data with various different parameters.

In [ ]:
def fake_spectrum(wleft, wright, si_all_val, bkg=0.09, err=0.007, dshift=0.0,
                  sample_amp=False, sample_width=False, sample_signs=False,
                  amp_hypermean=None, amp_hypersigma=0.08, nzero=0, 
                  width_hypermean=0.01, width_hypersigma=0.01):
    """
    Make a fake spectrum with emission/absorption lines.
    The background is constant, though that should later become an OU process or 
    something similar.
    
    NOTE: The amplitude *must not* fall below zero! I'm not entirely sure how to deal 
    with that yet!
    
    Parameters
    ----------
    wleft: np.ndarray
        Left edges of the energy bins
        
    wright: np.ndarray
        Right edges of the energy bins
    
    si_all_val: np.ndarray
        The positions of the line centroids
    
    bkg: float
        The value of the constant background
        
    err: float
        The width of the Gaussian error distribution
        
    dshift: float, default 0.0
        The Doppler shift of the spectral lines.
        
    sample_amp: bool, default False
        Sample all amplitudes? If not, whatever value is set in 
        `amp_hypermean` will be set as collective amplitude for all 
        lines
        
    sample_width: bool, default False
        Sample all line widths?  If not, whatever value is set in 
        `width_hypersigma` will be set as collective amplitude for all 
        lines
    
    sample_signs: bool, default False
        Sample the sign of the line amplitude (i.e. whether the line is an 
        absorption or emission line)? If False, all lines will be absorption 
        lines
        
    amp_hypermean: {float | None}, default None
        The mean of the Gaussian prior distribution on the line amplitude. If None, 
        it is set to the same value as `bkg`.
        
    amp_hypersigma: float, default 0.08
        The width of the Gaussian prior distribution on the line amplitude. 
        
    nzero: int, default 0
        The number of lines to set to zero amplitude
        
    width_hypermean: float, default 0.01
        The mean of the Gaussian prior on the line width
        
    width_hypersigma: float, default 0.01
        The width of the Gaussian prior distribution on the line width
    
    Returns
    -------
    model_flux: np.ndarray
        The array of model line fluxes for each bin
        
    fake_flux: np.ndarray
        The array of fake fluxes (with errors) for each bin
        
    """
    
    # number of lines
    nlines = si_all_val.shape[0]
    
    print("dshift: %.11f"%dshift)
    # shift spectral lines
    si_all_val_shifted = si_all_val*(1. + dshift)
    
    if sample_amp:  
        # sample line amplitudes
        amps = np.random.normal(amp_hypermean, amp_hypersigma, size=nlines)
    else:
        amps = np.zeros(nlines)+amp_hypermean
        
        
    if nzero > 0:
        zero_ind = np.random.choice(np.arange(nlines), size=nzero)
        for z in zero_ind:
            amps[int(z)] = 0.0
    
    if sample_signs:
        # sample sign of the amplitudes
        signs = np.random.choice([-1., 1.], p=[0.5, 0.5], size=nlines)
    else:
        # all lines are absorption lines
        signs = -1.*np.ones(nlines)
        
    # include signs in the amplitudes
    amps *= signs
    
    if sample_width:
        # widths of the lines
        widths = np.random.normal(width_hypermean, width_hypersigma, size=nlines)
    else:
        widths = np.ones(nlines)*width_hypermean
        
        
    model_flux = np.zeros_like(wleft) + bkg
    
    for si, a, w in zip(si_all_val_shifted, amps, widths):
        #print(a)
        model_flux += spectral_line(wleft, wright, si, a, w)
      
    fake_flux = model_flux + np.random.normal(0.0, 0.007, size=model_flux.shape[0])
    return model_flux, fake_flux
    
    

In [ ]:
for s in si_all_val:
    print("%.15f"%s)


### A non-shifted spectrum with all the same amplitudes

We'll make a set of spectra that are not Doppler shifted at all, where all lines have 
the same width and amplitude as a first dumb check of whether our method works.
All lines are absorption lines.

First spectrum: absorption spectrum with all lines absorption lines with the same height and width, no redshift:

In [ ]:
model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, amp_hypermean=0.3)

In [ ]:
#plt.errorbar(wnew_mid, cnew, yerr=enew, fmt="o")
plt.plot(wnew_mid, cnew, alpha=0.7)
plt.gca().invert_xaxis()
plt.xlim(wnew_mid[-1], wnew_mid[0])
#for s in si_all_angstrom:
#    plt.vlines(s.value, np.min(counts), np.max(counts), lw=2)

plt.plot(wnew_mid, fake_flux, alpha=0.7)

In [ ]:
for s in si_all_val:
    print(s/0.01)

Looks about right! Let's save that and a few other amplitudes:

In [ ]:
froot = "test_spectra_noshift_sameamp_samewidth"

amps = np.array([0.01, 0.1, 0.3, 0.5])
all_model_fluxes, all_fake_fluxes = [], []

for i,a in enumerate(amps):
    model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, amp_hypermean=a)
    all_model_fluxes.append(model_flux)
    all_fake_fluxes.append(fake_flux)
    np.savetxt(datadir+froot+"%i.txt"%i, 
               np.array([wnew_left, wnew_right, wnew_mid, fake_flux, model_flux]).T,
               header="# dshift=0.0, err=0.007, amp=%.3f, width=0.01"%a)

Let's see whether it worked:

In [ ]:
for i in range(amps.shape[0]):
    plt.figure(figsize=(16,6))
    data = np.loadtxt(datadir+froot+"%i.txt"%i)
    plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
    plt.plot(data[:,2], data[:,4])

Cool! So that worked!

Let's set some lines to zero:

In [ ]:
froot = "test_spectra_noshift_sameamp_samewidth_nzero2"

amp = 0.3

model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, 
                                      amp_hypermean=amp, nzero=2, sample_signs=False)

flux_err = np.ones_like(fake_flux)*0.007

np.savetxt(datadir+froot+".txt", 
           np.array([wnew_left, wnew_right, wnew_mid, fake_flux, model_flux]).T,
           header="# dshift=0.0, err=0.007, amp=%.3f, width=0.01"%a)

In [ ]:
plt.figure(figsize=(16,6))
data = np.loadtxt(datadir+froot+".txt")
plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
plt.plot(data[:,2], data[:,4])

In [ ]:
np.savetxt(datadir+"test_noshift3.dat", 
           np.array([wnew_left, wnew_right, fake_flux, flux_err]).T)

Same thing but with possible absorption and emission lines:

In [ ]:
froot = "test_spectra_noshift_sameamp_samewidth_emab"

amps = np.array([0.01, 0.1, 0.3, 0.5])
all_model_fluxes, all_fake_fluxes = [], []

for i,a in enumerate(amps):
    model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, amp_hypermean=a, sample_signs=True)
    all_model_fluxes.append(model_flux)
    all_fake_fluxes.append(fake_flux)
    np.savetxt(datadir+froot+"%i.txt"%i, 
               np.array([wnew_left, wnew_right, wnew_mid, fake_flux, model_flux]).T,
               header="# dshift=0.0, err=0.007, amp=%.3f, width=0.01"%a)

In [ ]:
for i in range(amps.shape[0]):
    plt.figure(figsize=(16,6))
    data = np.loadtxt(datadir+froot+"%i.txt"%i)
    plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
    plt.plot(data[:,2], data[:,4])

Awesomeness!

## A shifted spectrum with the same amplitudes and widths

Let's now add a Doppler shift to the spectrum:

In [ ]:
froot = "test_spectra_sameamp_samewidth"

amps = np.array([0.01, 0.1, 0.3, 0.5])
dshifts = np.array([0.001, 0.005, 0.01, 0.05, 0.1])
all_model_fluxes, all_fake_fluxes = [], []

for i,a in enumerate(amps):
    for j,d in enumerate(dshifts):
        print("a: " + str(a))
        print("d: " + str(d))
        model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, dshift=d,
                                              amp_hypermean=a, sample_signs=False)

        all_model_fluxes.append(model_flux)
        all_fake_fluxes.append(fake_flux)
        np.savetxt(datadir+froot+"_dshift=%.3f_%i.txt"%(d,i), 
                   np.array([wnew_left, wnew_right, wnew_mid, fake_flux, model_flux]).T,
                   header="# dshift=%.3f, err=0.007, amp=%.3f, width=0.01"%(d,a))

In [ ]:
for i in range(amps.shape[0]):
    for j,d in enumerate(dshifts):
        plt.figure(figsize=(16,6))
        data = np.loadtxt(datadir+froot+"_dshift=%.3f_%i.txt"%(d,i))
        plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
        plt.plot(data[:,2], data[:,4])

In [ ]:
plt.figure(figsize=(16,6))
data = np.loadtxt(datadir+"test_spectra_sameamp_samewidth_dshift=0.005_2.txt")
plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
plt.plot(data[:,2], data[:,4])

In [ ]:
np.savetxt(datadir+"test_shift1.dat", 
           np.array([data[:,0], data[:,1], data[:,3], flux_err]).T)

Same thing with both emission and absorption lines:

In [ ]:
froot = "test_shift2"

amp = 0.3

model_flux, fake_flux = fake_spectrum(wnew_left, wnew_right, si_all_val, 
                                      amp_hypermean=amp, nzero=0, sample_signs=True,
                                     dshift=0.005)

flux_err = np.ones_like(fake_flux)*0.007

np.savetxt(datadir+froot+"_withmodel.dat", 
           np.array([wnew_left, wnew_right, wnew_mid, fake_flux, model_flux]).T,
           header="# dshift=0.005, err=0.007, amp=%.3f, width=0.01"%amp)



In [ ]:
plt.figure(figsize=(16,6))
data = np.loadtxt(datadir+"test_shift2_withmodel.dat")
plt.errorbar(data[:,2], data[:,3], yerr=(data[:,3]-data[:,4]), fmt="o")
plt.plot(data[:,2], data[:,4])

In [ ]:
np.savetxt(datadir+"test_shift2.dat", 
           np.array([data[:,0], data[:,1], data[:,3], flux_err]).T)

## A simple model for non-Doppler shifted data

Let's first write a simple model that deals with non-Doppler shifted data:


In [ ]:
logmin = -1.e16

def gaussian_cdf(x, w0, width):
    return 0.5*(1. + scipy.special.erf((x-w0)/(width*np.sqrt(2.))))

def spectral_line(wleft, wright, w0, amplitude, width):
    """
    Use the CDF of a Gaussian distribution to define spectral 
    lines. We use the CDF to integrate over the energy bins, 
    rather than taking the mid-bin energy.
    
    Parameters
    ----------
    wleft: array
        Left edges of the energy bins
        
    wright: array
        Right edges of the energy bins
        
    w0: float
        The centroid of the line
        
    amplitude: float
        The amplitude of the line
        
    width: float
        The width of the line
        
    Returns
    -------
    line_flux: array
        The array of line fluxes integrated over each bin
    """
    line_flux = amplitude*(gaussian_cdf(wright, w0, width)-
                           gaussian_cdf(wleft, w0, width))
    return line_flux

class LinePosterior(object):
    def __init__(self, x_left, x_right, y, yerr, line_pos):
        """
        A class to compute the posterior of all the lines in 
        a spectrum.
        
        Parameters
        ----------
        x_left: np.ndarray
            The left edges of the independent variable (wavelength bins)
        
        x_right: np.ndarray
            The right edges of the independent variable (wavelength bins)
            
        y: np.ndarray
            The dependent variable (flux)
            
        yerr: np.ndarray
            The uncertainty on the dependent variable (flux)
            
        line_pos: np.ndarray
            The rest-frame positions of the spectral lines
            
        Attributes
        ----------
        x_left: np.ndarray
            The left edges of the independent variable (wavelength bins)
        
        x_right: np.ndarray
            The right edges of the independent variable (wavelength bins)
          
        x_mid: np.ndarray
            The mid-bin positions
            
        y: np.ndarray
            The dependent variable (flux)
            
        yerr: np.ndarray
            The uncertainty on the dependent variable (flux)
            
        line_pos: np.ndarray
            The rest-frame positions of the spectral lines

        nlines: int
            The number of lines in the model
        
        """
    
        self.x_left = x_left
        self.x_right = x_right
        self.x_mid = x_left + (x_right-x_left)/2.
        
        self.y = y
        
        assert np.size(yerr) == 1, "Multiple errors are not supported!"
        
        self.yerr = yerr
        self.line_pos = line_pos
        self.nlines = len(line_pos)
        
    def logprior(self, t0):
        """
        The prior of the model. Currently there are Gaussian priors on the 
        line width as well as the amplitude and the redshift.
    
        Parameters
        ----------
        t0: iterable
            The list or array with the parameters of the model
            Contains:
                * Doppler shift
                * a background parameter
                * all line amplitudes
                * all line widths

            
        Returns
        -------
        logp: float
            The log-prior of the model
        
        """
        # t0 must have twice the number of lines (amplitude, width for each) plus a 
        # background plus the redshift
        assert len(t0) == 2*self.nlines+2, "Wrong number of parameters!"
        
        # get out the individual parameters
        dshift = t0[0] # Doppler shift
        log_bkg = t0[1]
        amps = t0[2:2+self.nlines]
        log_widths = t0[2+self.nlines:]
        
        # prior on the Doppler shift is Gaussian
        dshift_hypermean = 0.0
        dshift_hypersigma = 0.01
        
        dshift_prior = scipy.stats.norm(dshift_hypermean, dshift_hypersigma)
        p_dshift = np.log(dshift_prior.pdf(dshift))
        
        #print("p_dshift: " + str(p_dshift))
        
        # Prior on the background is uniform
        logbkg_min = -10.0
        logbkg_max = 10.0
        p_bkg = (log_bkg >= logbkg_min and log_bkg <= logbkg_max)/(logbkg_max-logbkg_min)
        if p_bkg == 0:
            p_logbkg = logmin
        else:
            p_logbkg = 0.0
        
        #print("p_logbkg: " + str(p_logbkg))
        
        # prior on the amplitude is Gaussian
        amp_hypermean = 0.0
        amp_hypersigma = 0.1
        
        amp_prior = scipy.stats.norm(amp_hypermean, amp_hypersigma)
        
        p_amp = 0.0
        for a in amps:
            p_amp += np.log(amp_prior.pdf(a))
        
        #print("p_amp: " + str(p_amp))
        # prior on the log-widths is uniform:
        logwidth_min = -5.
        logwidth_max = 3.
        
        p_logwidths = 0.0
        for w in log_widths:
            #print("w: " + str(w))
            p_width = (w >= logwidth_min and w <= logwidth_max)/(logwidth_max-logwidth_min)
            if p_width == 0.0:
                p_logwidths += logmin
            else:
                continue
                
        #print("p_logwidths: " + str(p_logwidths))
        logp = p_dshift + p_logbkg + p_amp + p_logwidths
        
        return logp
    
    @staticmethod
    def _spectral_model(x_left, x_right, dshift, logbkg, line_pos, amplitudes, logwidths):
        """
        The spectral model underlying the data. It uses the object 
        attributes `x_left` and `x_right` to integrate over the bins 
        correctly.
        
        Parameters
        ----------
        x_left: np.ndarray
            The left bin edges of the ordinate (wavelength bins)
            
        x_right: np.ndarray
            The right bin edges of the ordinate (wavelength bins)
            
        dshift: float
            The Doppler shift
            
        logbkg: float
            Logarithm of the constant background level
        
        line_pos: iterable
            The rest frame positions of the line centroids in the same 
            units as `x_left` and `x_right`
        
        amplitudes: iterable
            The list of all line amplitudes
            
        logwidths: iterable
            The list of the logarithm of all line widths
        
        Returns
        -------
        flux: np.ndarray
            The integrated flux in the bins defined by `x_left` and `x_right`
            
        """
        
        assert len(line_pos) == len(amplitudes), "Line positions and amplitudes must have same length"
        assert len(line_pos) == len(logwidths), "Line positions and widths must have same length"
        
        # shift the line position by the redshift
        line_pos_shifted = line_pos + dshift
        
        #print(line_pos_shifted)
        # exponentiate logarithmic quantities
        bkg = np.exp(logbkg)
        widths = np.exp(logwidths)
        
        #print(widths)
        # background flux
        flux = np.zeros_like(x_left) + bkg

        #print(amplitudes)
        
        # add all the line fluxes
        for x0, a, w in zip(line_pos_shifted, amplitudes, widths):
            flux += spectral_line(x_left, x_right, x0, a, w)

        return flux

        
        
    def loglikelihood(self, t0):
        """
        The Gaussian likelihood of the model. 
    
        Parameters
        ----------
        t0: iterable
            The list or array with the parameters of the model
            Contains:
                * Doppler shift
                * a background parameter
                * all line amplitudes
                * all line widths

            
        Returns
        -------
        loglike: float
            The log-likelihood of the model
        
        """
        
        assert len(t0) == 2*self.nlines+2, "Wrong number of parameters!"
        
        # get out the individual parameters
        dshift = t0[0] # Doppler shift
        logbkg = t0[1]
        amplitudes = t0[2:2+self.nlines]
        logwidths = t0[2+self.nlines:]

        model_flux = self._spectral_model(self.x_left, self.x_right, 
                                          dshift, logbkg, self.line_pos, 
                                          amplitudes, logwidths)

        loglike = -(len(self.y)/2.)*np.log(2.*np.pi*self.yerr**2.) - \
                    np.sum((self.y-model_flux)**2./(2.*self.yerr**2.))
            
        return loglike
    
    def logposterior(self, t0):
        """
        The Gaussian likelihood of the model. 
    
        Parameters
        ----------
        t0: iterable
            The list or array with the parameters of the model
            Contains:
                * Doppler shift
                * a background parameter
                * all line amplitudes
                * all line widths

            
        Returns
        -------
        logpost: float
            The log-likelihood of the model
        
        """

        
        # assert the number of input parameters is correct:
        assert len(t0) == 2*self.nlines+2, "Wrong number of parameters!"

        logpost = self.logprior(t0) + self.loglikelihood(t0)
        #print("prior: " + str(self.logprior(t0)))
        #print("likelihood: " + str(self.loglikelihood(t0)))
        #print("posterior: " + str(self.logposterior(t0)))
        if np.isfinite(logpost):
            return logpost
        else:
            return logmin

    def __call__(self, t0):
        return self.logposterior(t0)

Now we can use `emcee` to sample. 

We're going to use one of our example data sets, one without Doppler shift and strong lines:

In [ ]:
data = np.loadtxt(datadir+"test_spectra_noshift_sameamp_samewidth3.txt")

x_left = data[:,0]
x_right = data[:,1]
flux = data[:,3]
f_err = 0.007

lpost = LinePosterior(x_left, x_right, flux, f_err, si_all_val)

plt.errorbar(lpost.x_mid, flux, yerr=f_err, fmt="o", label="Fake data")
plt.plot(lpost.x_mid, data[:,4], label="Underlying model")


In [ ]:
d_test = 0.0
bkg_test = np.log(0.09)
amp_test = np.zeros_like(si_all_val) - 0.5
logwidth_test = np.log(np.zeros_like(si_all_val) + 0.01)

p_test = np.hstack([d_test, bkg_test, amp_test, logwidth_test])

In [ ]:
lpost.logprior(p_test)

In [ ]:
d_test = 0.0
bkg_test = np.log(0.09)
amp_test = np.zeros_like(si_all_val) - 0.5
logwidth_test = np.log(np.zeros_like(si_all_val) + 0.01)

p_test = np.hstack([d_test, bkg_test, amp_test, logwidth_test])

lpost.loglikelihood(p_test)

In [ ]:
nwalkers = 1000
niter = 300
burnin = 300

# starting positions for all parameters, from the prior
# the values are taken from the `logprior` method in `LinePosterior`.
# If the hyperparameters of the prior change in there, they'd better 
# change here, too!
dshift_start = np.random.normal(0.0, 0.01, size=nwalkers)
logbkg_start = np.random.uniform(-10., 10., size=nwalkers)
amp_start = np.random.normal(0.0, 0.1, size=(lpost.nlines, nwalkers))
logwidth_start = np.random.uniform(-5., 3., size=(lpost.nlines, nwalkers))
p0 = np.vstack([dshift_start, logbkg_start, amp_start, logwidth_start]).T

ndim = p0.shape[1]

In [ ]:

sampler = emcee.EnsembleSampler(nwalkers, ndim, lpost)
pos, prob, state = sampler.run_mcmc(p0, burnin)


In [ ]:
sampler.reset()

## do the actual MCMC run
pos, prob, state = sampler.run_mcmc(pos, niter, rstate0=state)


In [ ]:
mcall = sampler.flatchain

In [ ]:
mcall.shape

In [ ]:
for i in range(mcall.shape[1]):
    pmean = np.mean(mcall[:,i])
    pstd = np.std(mcall[:,i])
    print("Parameter %i: %.4f +/- %.4f"%(i, pmean, pstd))

In [ ]:
mcall.shape

In [ ]:
corner.corner(mcall);

In [ ]:
lpost = LinePosterior(x_left, x_right, flux, f_err, si_all_val)

plt.errorbar(lpost.x_mid, flux, yerr=f_err, fmt="o", label="Fake data")
plt.plot(lpost.x_mid, data[:,4], label="Underlying model")

randind = np.random.choice(np.arange(mcall.shape[0]), replace=False, size=20)
for ri in randind:
    ri = int(ri)
    p = mcall[ri]
    dshift = p[0]
    logbkg = p[1]
    line_pos = lpost.line_pos
    amplitudes = p[2:2+lpost.nlines]
    logwidths = p[2+lpost.nlines:] 
    plt.plot(lpost.x_mid, lpost._spectral_model(x_left, x_right, dshift, logbkg, line_pos,
                                               amplitudes, logwidths))

## Preparing Data for DNest4 run

The data as I saved it can't be read into DNest4, so I need to save it into a better format:

In [ ]:
data = np.loadtxt(datadir+"test_spectra_noshift_sameamp_samewidth3.txt")

In [ ]:
x_left = data[:,0]
x_right = data[:,1]

y = data[:,3]
yerr = np.ones_like(y)*0.007

np.savetxt("/Users/danielahuppenkothen/work/repositories/shiftylines/data/test_noshift1.dat",
          np.array([x_left, x_right, y, yerr]).T)

In [ ]:
data = np.loadtxt("/Users/danielahuppenkothen/work/repositories/shiftylines/data/test_noshift1.dat")
data.shape

### Looking at the results

Let's look at some initial results:

In [ ]:
dnest_dir = "/Users/danielahuppenkothen/work/repositories/shiftylines/code/"

samples = np.atleast_2d(np.loadtxt(dnest_dir+"posterior_sample.txt"))

In [ ]:
plt.plot(x_left, samples[0,-x_left.shape[0]:])

In [ ]:
s = samples[0]

## Real Data

Let's look at some real data!

In [ ]:
data = np.loadtxt("/Users/danielahuppenkothen/work/data/spectral_data/shifty_lines/8525_nodip_cut.txt")

In [ ]:
plt.errorbar(data[:,0], data[:,3], yerr=data[:,4], fmt="o")

In [ ]:
np.savetxt("/Users/danielahuppenkothen/work/repositories/shiftylines/data/cygx1_chandra1.dat", np.array([data[:,0], data[:,1], data[:,3], data[:,4]]).T)

Things to think about:
* what if we don't have ten lines, but a hundred?
* want to do actual inference on individual lines (i.e. want to know where they are)
* how do we incorporate errors in the line positions?
* should we do a straight model comparison for the redshifts?
* how do we incorporate sections of data?